In [1]:
import cartoframes
import pandas as pd
import numpy as np

APIKEY = '678b44cb2affbc52ea3e319f9e2a0d73ac86623c'
BASEURL = 'https://iccmtest.carto.com/'

# cc = cartoframes.CartoContext(api_key=APIKEY, base_url=BASEURL, verbose=True)
cc = cartoframes.CartoContext(api_key='7db1d0957a6f4a10c9ab8bd437ed6b3e28b3d960',
                              base_url='https://eschbacher.carto.com/',
                              verbose=False)

In [2]:
from cartoframes import Layer, QueryLayer

In [3]:
l = Layer('watermelon')

In [3]:
N = 17000
data = {'id': range(N),
        'lat': 2. * np.random.randn(N),
        'long': 2. * np.random.randn(N),
        'outside_range': abs(np.random.randn(N)) > 1,
        'new_id': list('a{}'.format(i) for i in range(N))}

df = pd.DataFrame(data)

def print_me(row):
    p_text = 'Point ({0} {1})'.format(row['long'], row['lat'])
    return p_text

df['the_geom'] = df.apply(print_me, axis=1)

In [3]:
df.head()

,id,lat,long,new_id,outside_range,the_geom
0,0,1.015654,-2.982029,a0,False,Point (-2.982028758797373 1.0156535292587272)
1,1,0.441646,1.073033,a1,True,Point (1.0730334543370934 0.4416458130963483)
2,2,0.633870,2.170501,a2,True,Point (2.1705012509644934 0.6338695850680947)
3,3,1.197145,-1.550228,a3,False,Point (-1.5502283081342452 1.197144582690541)
4,4,-0.101693,0.377303,a4,True,Point (0.37730346734896647 -0.10169294799027497)


In [6]:
df.dtypes

id                 int64
lat              float64
long             float64
new_id            object
outside_range       bool
the_geom          object
dtype: object

In [4]:
df.shape

(1700000, 6)

In [4]:
cc.write(df, 'cartoframes_upload_test3',
         overwrite=True)

The following columns were renamed because of PostgreSQL column normalization requirements: `field_1`, `the_geom_webmercator`
Table written to CARTO: https://eschbacher.carto.com/dataset/cartoframes_upload_test3


In [11]:
df2 = cc.read('cartoframes_upload_test2')

In [12]:
df2.head()

,the_geom,the_geom_webmercator,field_1,id,lat,long,new_id,outside_range
cartodb_id,,,,,,,,
3998,0101000020E61000003ACF7730657E1040F55D49C58BAA...,0101000020110F00000C8F79CB29041C41D63ECA0249CF...,3997,3997.0,3.208274,4.123433,a3997,True
1400,0101000020E6100000F925E46DFED8DBBFCA83A2BF6126...,0101000020110F0000A7E66B1FA8A6E7C096912D1CD8F6...,1399,1399.0,1.321871,-0.435119,a1399,False
1588,0101000020E6100000A56834E2B5A400409F95458522AD...,0101000020110F0000F79A5ED93E450C418AA16B997CBB...,1587,1587.0,-2.334538,2.080425,a1587,False
2594,0101000020E610000025E79EBFC8CCF53F325F86E36039...,0101000020110F0000949C3B02C3830241FAC14FDF0854...,2593,2593.0,-1.201508,1.362496,a2593,False
3205,0101000020E61000008EF53BD812E8F83FE7923ACA72D0...,0101000020110F00002462DF89332705410AAD09B17806...,3204,3204.0,1.988391,1.556659,a3204,False


In [4]:
from cartoframes import Layer, BaseMap, QueryLayer
cc.map(layers=[BaseMap('dark', labels=None),
               Layer('cartoframes_upload_test2'),
               QueryLayer('select * from cartoframes_upload_test2 where random() < 0.25'),
               QueryLayer('select * from cartoframes_upload_test2 where random() < 0.0625'),
               QueryLayer('select * from cartoframes_upload_test2 where random() < 0.015625'),
               QueryLayer('select * from cartoframes_upload_test2 where random() < 0.00390625')],
       interactive=False)

In [2]:
from cartoframes import Layer, BaseMap, QueryLayer, styling
cc.map(layers=[BaseMap('dark', labels=None),
               QueryLayer('select * from all_month_3 where random() < 0.25',
                          color={'column': 'net', 'scheme': styling.vivid(7)})],
       interactive=False)

In [8]:
df.columns

Index(['the_geom', 'the_geom_webmercator', 'description', 'amount',
       'amt_healthcare', 'amt_person', 'amt_transaction', 'card_account_id',
       'card_transaction_id', 'chain_name', 'city', 'cobrand_id',
       'confidence_score', 'currency_id', 'factual_category', 'factual_id',
       'fax_number', 'file_created_date', 'latitude', 'longitude', 'mem_id',
       'merchant_name', 'neighbourhood', 'phone_number', 'post_date', 'state',
       'store_id', 'street', 'sum_info_id', 'transaction_base_type',
       'transaction_category_name', 'transaction_date', 'transaction_origin',
       'transaction_src_element_id', 'transactions_person',
       'unique_card_account_id', 'unique_card_transaction_id', 'unique_mem_id',
       'website', 'zip_code', 'transaction_category_id'],
      dtype='object')

In [9]:
df = cc.read('yodlee_points',
             limit=1)
cols = set(df.columns) - {'the_geom_webmercator'}
N = 800000
df = cc.query('''select {cols} 
                 from yodlee_points
                 limit {n}'''.format(cols=','.join(cols),
                                     n=N))

In [10]:
df.head()

,city,amount,factual_category,post_date,longitude,file_created_date,latitude,card_account_id,transaction_base_type,website,...,transaction_date,cobrand_id,transaction_src_element_id,amt_healthcare,sum_info_id,amt_person,neighbourhood,transaction_origin,amt_transaction,the_geom
0,Newark,50.00,"[""Transportation""]",2015-07-18,-74.180278,2015-07-22,40.721478,14357345.0,debit,http://www.ezpass.com,...,2015-07-18,1.000564e+07,-75546255,0.0,16455.0,0.0,"[""Newark North"",""South Broad Street""]",Physical,0.0,0104000020E61000000100000001010000000D8CBCAC89...
1,Manhasset,55.00,"[""Services and Supplies"",""Personal Care"",""Beau...",2015-05-02,-73.691851,2015-07-22,40.793338,4347042.0,debit,http://www.nubestsalon.com,...,2015-05-02,1.000616e+07,765741146,0.0,19267.0,0.0,,Physical,0.0,0104000020E61000000100000001010000001DAD6A4947...
2,Clifton,58.74,"[""Social"",""Food and Dining"",""Restaurants"",""Del...",2015-07-16,-74.173989,2015-07-22,40.894212,41936783.0,debit,,...,2015-07-14,1.000137e+07,1620544585,0.0,12847.0,0.0,"[""Great Notch""]",Physical,0.0,0104000020E61000000100000001010000004837C2A222...
3,Englewood Cliffs,34.29,,2015-07-22,-73.947418,2015-07-22,40.881950,1195961.0,debit,,...,2015-07-20,1.000616e+07,1646705609,0.0,19267.0,0.0,,Physical,0.0,0104000020E610000001000000010100000012691B7FA2...
4,Staten Island,9.01,,NaT,-74.065463,2015-07-22,40.598220,12317381.0,debit,,...,2015-07-16,2.125103e+09,743737414,0.0,1503.0,0.0,"[""Fort Wadsworth""]",Physical,0.0,0104000020E61000000100000001010000004606B98B30...


In [11]:
cc.write(df, 'yodlee_subset_cartoframes_upload3',
         overwrite=True)

Uploading in batches: : 100%|██████████| 2/2 [13:08<00:00, 483.86s/it]


The following columns were renamed because of PostgreSQL column normalization requirements: `the_geom_webmercator`
Table written to CARTO: https://eschbacher.carto.com/dataset/yodlee_subset_cartoframes_upload3


In [6]:
import requests
import binascii as ba
from shapely import wkb, wkt

In [19]:
p = 'Point (-122 38)'
ba.hexlify(wkb.dumps(wkt.loads(p))).decode()

'01010000000000000000805ec00000000000004340'

In [4]:
df_t = cc.read('all_month_3', decode_geom=True)

In [5]:
df_t.the_geom[1]

'0101000020E6100000F7F2F1BF4B775EC0B4C068A0432B4440'

In [14]:
cc.api_key

'7db1d0957a6f4a10c9ab8bd437ed6b3e28b3d960'

In [20]:
df = pd.read_csv('taxi_smallest.csv')
df.head()

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,303227,2,2016-05-01 14:52:11,2016-05-01 15:00:36,2,2.08,-74.006706,40.730461,1.0,N,-74.012383,40.706779,1.0,8.5,0.0,0.5,1.00,0.0,0.3,10.30
1,183993,1,2016-05-01 08:34:08,2016-05-01 08:49:02,1,3.00,-73.924957,40.744125,1.0,N,-73.973824,40.762779,1.0,13.5,0.0,0.5,2.00,0.0,0.3,16.30
2,1434785,1,2016-05-04 09:44:40,2016-05-04 10:07:09,1,2.10,-73.973488,40.748501,1.0,N,-73.998955,40.740833,2.0,14.5,0.0,0.5,0.00,0.0,0.3,15.30
3,421142,2,2016-05-01 20:50:11,2016-05-01 21:05:24,1,4.41,-73.999786,40.743267,1.0,N,-73.966362,40.792370,2.0,15.0,0.5,0.5,0.00,0.0,0.3,16.30
4,547655,2,2016-05-02 07:26:56,2016-05-02 07:53:53,2,4.01,-73.963631,40.803360,1.0,N,-73.956963,40.784939,1.0,19.5,0.0,0.5,4.06,0.0,0.3,24.36


In [43]:
def print_me(row):
    p_text = 'Point ({0} {1})'.format(row['pickup_longitude'], row['pickup_latitude'])
    return ba.hexlify(wkb.dumps(wkt.loads(p_text))).decode()

df['the_geom'] = df.apply(print_me, axis=1)

In [44]:
df.head()

,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,...,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,the_geom
0,303227,2,2016-05-01 14:52:11,2016-05-01 15:00:36,2,2.08,-74.006706,40.730461,1.0,N,...,40.706779,1.0,8.5,0.0,0.5,1.00,0.0,0.3,10.30,0101000000010000e06d8052c0000000c07f5d4440
1,183993,1,2016-05-01 08:34:08,2016-05-01 08:49:02,1,3.00,-73.924957,40.744125,1.0,N,...,40.762779,1.0,13.5,0.0,0.5,2.00,0.0,0.3,16.30,010100000000000080327b52c0000000803f5f4440
2,1434785,1,2016-05-04 09:44:40,2016-05-04 10:07:09,1,2.10,-73.973488,40.748501,1.0,N,...,40.740833,2.0,14.5,0.0,0.5,0.00,0.0,0.3,15.30,0101000000000000a04d7e52c0000000e0ce5f4440
3,421142,2,2016-05-01 20:50:11,2016-05-01 21:05:24,1,4.41,-73.999786,40.743267,1.0,N,...,40.792370,2.0,15.0,0.5,0.5,0.00,0.0,0.3,16.30,010100000000000080fc7f52c000000060235f4440
4,547655,2,2016-05-02 07:26:56,2016-05-02 07:53:53,2,4.01,-73.963631,40.803360,1.0,N,...,40.784939,1.0,19.5,0.0,0.5,4.06,0.0,0.3,24.36,010100000001000020ac7d52c000000080d4664440


In [45]:
df.to_csv('test_geom_table.csv')

In [8]:
import requests
with open('test_geom_table.csv') as f:
    resp = requests.post('https://eschbacher.carto.com/api/v1/imports?api_key=7db1d0957a6f4a10c9ab8bd437ed6b3e28b3d960', 
                         files={'file': f},
                         params={'type_guessing': 'false',
                                 'table_name': 'testing_content_guessing_false'},
                         stream=True)


In [9]:
resp.text

'{"item_queue_id":"2368ce5c-6df2-4f6f-921e-a4901ec61221","success":true}'